In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import datetime

In [ ]:
def read_var(varname, filename):
    with open(filename, 'r') as f:
        data = f.readlines()

    for (i,x) in enumerate(data):
        if ' ' + varname in x: start = i
                
    end = i
    for (i,x) in enumerate(data[start:]):
        if not x.strip(): lenblock = i; break
    var = pd.read_fwf(filename, header = None, squeeze = True, sep=' ',infer_nrows=lenblock, skiprows=start+1, nrows=lenblock)
    return var

In [ ]:
def to_xr(var):
    var.to_xarray()

In [ ]:
def filesanddates():    
    filename = "/Users/tobias/files.txt"
    with open(filename, 'r') as f:
        data = f.readlines()
    
    years = [int(d[0:4]) for d in data]
    months = [(d[5:7]) for d in data]
    monthss = ['DJ','JF','FM','MA','AM','MJ','JJ','JA','AS','SO','ON','ND']
    mon = [monthss.index(m) for m in months]
    test = [d[0:7] for d in data]
    date = [datetime.datetime(years[i], mon[i]+1, 1) for i in range(len(years))]
    
# filename = '/Users/tobias/Desktop/QBO Ancistrus/test10a.dat'
#    years = range(2002,2013)
#    year = [str(y) for y in range(2002,2013)]
#    months = ['DJ','JF','FM','MA','AM','MJ','JJ','JA','AS','SO','ON','ND']
#    month = [str(m) for m in months]
#    test = []
#    date = []
#    for y in year:
#        for m in month:
#            test.append( y + '_' + m )
#            date.append(datetime.datetime(years[year.index(y)], month.index(m)+1, 1))
#    
#    test = np.delete(test, [0,1,2,3,4,5,6,27,28,29,30,31,32,33,34,35,-8,-7,-6,-5,-4,-3,-2,-1])
#    date = np.delete(date, [0,1,2,3,4,5,6,27,28,29,30,31,32,33,34,35,-8,-7,-6,-5,-4,-3,-2,-1])
    return(test, date)

In [ ]:
def readvars(filename):
    varname = 'v_phi'
    var = read_var(varname, filename)

    varname = 'v_z'
    varz = read_var(varname, filename)

    varname = 'model altitude'
    alt = read_var(varname, filename)

    varname = 'model latitude'
    lat = read_var(varname, filename)

    latlist = lat.T.to_xarray().to_array().values.ravel()
    latlist = latlist[~np.isnan(latlist)]
    #print(latlist)
    altlist = alt.T.to_xarray().to_array().values.ravel()
    altlist = altlist[~np.isnan(altlist)]
    #print(altlist)
    varlist = var.to_xarray().to_array().values.ravel()
    varlist.resize(len(altlist),len(latlist))

    varzlist = varz.to_xarray().to_array().values.ravel()
    varzlist.resize(len(altlist),len(latlist))

    #print(np.shape(varlist),np.shape(var))
    
    varlist = varlist[np.newaxis,...]
    varzlist = varzlist[np.newaxis,...]
    return(varlist,varzlist,altlist,latlist)

In [ ]:
def readvars2(filename):
    varname = 'v_phi'
    var = read_var(varname, filename)

    varname = 'v_z'
    varz = read_var(varname, filename)

    varname = 'model altitude'
    alt = read_var(varname, filename)

    varname = 'model latitude'
    lat = read_var(varname, filename)

    latlist = lat.T.to_xarray().to_array().values.ravel()
    latlist = latlist[~np.isnan(latlist)]
    #print(latlist)
    altlist = alt.T.to_xarray().to_array().values.ravel()
    altlist = altlist[~np.isnan(altlist)]
    #print(altlist)
    varlist = var.to_xarray().to_array().values.ravel()
    varlist.resize(len(latlist),len(altlist))

    varzlist = varz.to_xarray().to_array().values.ravel()
    varzlist.resize(len(latlist),len(altlist))
    #print(np.shape(varlist),np.shape(var))
    
    varlist = varlist[np.newaxis,...]
    varzlist = varzlist[np.newaxis,...]
    return(varlist,varzlist,altlist,latlist)

In [ ]:
#df = pd.DataFrame({'year': [2014],
#...                    'month': [9],
#...                    'day': [6]})
#time = pd.to_datetime(df)

In [ ]:
def mkds(varlist,varzlist,latlist, altlist,time):# define data with variable attributes
    data_vars = {'v_phi':(['time', 'alt', 'lat'], varlist, 
                         {'units': 'm/s', 
                          'long_name':'meridional velocity'}),
                 'v_z'  :(['time', 'alt', 'lat'], varzlist, 
                         {'units': 'm/s', 
                          'long_name':'vertical velocity'})}

# define coordinates
    coords = {'time': (['time'], [time]),
         'lat': (['lat'], latlist),
         'alt': (['alt'], altlist)}

# define global attributes
    attrs = {'creation_date':str(datetime.datetime.now()), 
         'author':'Tobias Kerzenmacher', 
         'email':'kerzenmacher@kit.edu'}

# create dataset
    ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)
    return(ds)

In [ ]:
def mkds2(varlist,varzlist,latlist, altlist,time):# define data with variable attributes
    data_vars = {'v_phi':(['time', 'lat', 'alt'], varlist, 
                         {'units': 'm/s', 
                          'long_name':'meridional velocity'}),
                 'v_z'  :(['time', 'lat', 'alt'], varzlist, 
                         {'units': 'm/s', 
                          'long_name':'vertical velocity'})}

# define coordinates
    coords = {'time': (['time'], [time]),
         'lat': (['lat'], latlist),
         'alt': (['alt'], altlist)}

# define global attributes
    attrs = {'creation_date':str(datetime.datetime.now()), 
         'author':'Tobias Kerzenmacher', 
         'email':'kerzenmacher@kit.edu'}

# create dataset
    ds = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)
    return(ds)

In [ ]:
ds.to_netcdf('est.nc')

In [ ]:
import datetime

d = datetime.datetime(2019, 4, 13)
print(d)

In [ ]:
for t in test:
    print(t)
    print(date[test.index(t)])

In [ ]:
test, date = filesanddates()
#test=test[0:2]
#date=date[0:2]
for i,t in enumerate(test):
    filename = '/Users/tobias/Mount/Ancistrus/test10a_'+t+'/output/test10a.dat'
    time = date[i]
    print(filename,time,i)
    varlist, varzlist, altlist, latlist = readvars(filename)
    ds1 = mkds(varlist,varzlist,latlist, altlist,time)
    if i==0 : 
        ds = mkds(varlist,varzlist,latlist, altlist,time)
    else:
        ds= xr.concat([ds,mkds(varlist,varzlist,latlist, altlist,time)],dim='time')
    print(ds)


In [ ]:
test, date = filesanddates()
#test=test[0:2]
#date=date[0:2]
for i,t in enumerate(test):
    filename = '/Users/tobias/Mount/Ancistrus/test10a_'+t+'/output/test10a.dat'
    time = date[i]
    print(filename,time,i)
    varlist, varzlist, altlist, latlist = readvars2(filename)
    #ds1 = mkds2(varlist,varzlist,latlist, altlist,time)
    if i==0 : 
        ds2 = mkds2(varlist,varzlist,latlist, altlist,time)
    else:
        ds2= xr.concat([ds,mkds2(varlist,varzlist,latlist, altlist,time)],dim='time')
    print(ds)


In [ ]:
ds

In [ ]:
ds.v_z[2].plot()

In [ ]:
xr.plot.contour(ds.v_z[2])

In [ ]:
xr.plot.contour(ds2.v_z[2])